# Math Exam Generator

1. Provide a template using plain text.
  - Choose appropiate method to express placeholders for later replacement.
  - Maybe a kind of hierarchy will be needed to handle templates for questions and a template for the whole exam.
1. Establish a class that will control problem generation.
  - This class will have a function that will generate a whole question.
1. Establish a class that will have a list of problem instances. The class in itself will have a template that will serve to create the final whole exam.


**Status**: 
- Base class created. It is generating text correctly starting from a template file.
- Need a new class that can work with a list of problem generating classes.

## Prompt to create a base class for text generation.

Create a class called TextGenerator. This class will have:
1. A method to load a text file into an attribute called template. The loading should be also possible if a filename is provided on instance creation.
2. A method to parse the template and recognize the set of placeholders needed. Curly brackets will be used to designate the placeholders. The recognized placeholders will be stored in a dictionary called parameters.
3. An abstract method to compute placeholder values that will use the attribute parameters.
4. A method to generate the final string that will use the template and the parameters to replace the placeholders for actual values.

In [9]:
class TextGenerator:
    def __init__(self, filename=None):
        self.template = ""
        self.parameters = {}
        self.generated = ""
        if filename:
            self.load_template(filename)
            self.parse_template()

    def load_template(self, filename):
        with open(filename, "r") as f:
            self.template = f.read()

    def parse_template(self):
        import re
        pattern = re.compile(r"{\s*([a-zA-Z0-9_]+)\s*}")
        self.parameters = {match.group(1): None for match in pattern.finditer(self.template)}

    def compute_parameters(self):
        raise NotImplementedError

    def generate_text(self):
        for key, value in self.compute_parameters().items():
            if key in self.parameters:
                self.parameters[key] = value
        self.generated = self.template.format(**self.parameters)                
        return self.generated
    
    def save(self,filename='output.txt', filetype='txt'):
        if self.generated != "":
            if filetype.lower() == 'txt':
                with open(f'{filename}.{filetype}','w') as f:
                    f.write(self.generated)
            # Consider to add code to save txt2txt, txt2md and txt2pdf or something similar        
            # elif filetype.lower() == 'md':
            # elif filetype.lower() == 'pdf':
        else:
            print('Nothing to save.')


## Prompt

Considering the code below, create a new class called ExamGenerator that will inherit from TextGenerator. ExamGenerator will have an attribute items, a dictionary of TextGenerator instances. ExamenGenerator methods:
1. Add elements to items, it will need a name and a TextGenerator instace.
2. Remove element, that receives a name and removes corresponding pair from items.
3. Clear items.
4. compute_parameters will be override. It will call the generate_text method of every value of the dictionary items. It will return another dictionary of names and generated texts.
Code for TextGenerator:
Include code of TextGenerator.


In [10]:
class ExamGenerator(TextGenerator):
    def __init__(self, filename=None):
        super().__init__(filename)
        self.items = {}

    def add_item(self, name, text_generator):
        self.items[name] = text_generator

    def remove_item(self, name):
        del self.items[name]

    def clear_items(self):
        self.items.clear()

    def compute_parameters(self):
        generated_texts = {}
        for name, text_generator in self.items.items():
            generated_texts[name] = text_generator.generate_text()
        return generated_texts

## Prompt:

Create a new class called FormulaCuadratica. This class will inherit from TextGenerator. Overload the method compute_parameters with code that performs the following:
1. Using sympy define 3 symbols called a,b, and c. Assign to these symbols values taken randomly from a list of values that have numbers from -10 to 10 except zero.
2. Create symbol x.
3. Create the sympy expression called eq1 that is equivalent to this latex expression 'ax^2 = -bx-c'.
4.  Create the sympy expression called formula that is equivalent to this latex expression 'x=\frac{-b\pm \sqrt{b^2-4ac}}{2a}'
5. Create a local dictionary called tmp_parameters and store the values for a,b,c,eq1, and formula. In every case convert the value to a latex string.
6. Transfer the values from tmp_parameters to attribute parameters. Match corresponding keys, do not overwrite.

In [11]:
from random import choice
from sympy import symbols, latex, Eq, sqrt

class FormulaCuadratica(TextGenerator):
    def __init__(self, filename=None):
        super().__init__(filename)

    def compute_parameters(self):
        a, b, c = symbols('a b c')
        x = symbols('x')
        nonzero_values = [n for n in range(-10, 11) if n != 0]
        a_value, b_value, c_value = choice(nonzero_values), choice(nonzero_values), choice(nonzero_values)
        eq1 = Eq(a*x**2, -b*x-c)
        formula = Eq(x, (-b+sqrt(b**2-4*a*c))/(2*a))
        values = {a:a_value,
                  b:b_value,
                  c:c_value}  
        eq1_subs = eq1.subs(values)      
        tmp_parameters = {
            'eq1': latex(eq1_subs),
            'formula': latex(formula)
        }
        return tmp_parameters


In [12]:
p1 = FormulaCuadratica(filename='problem1.txt')
p2 = FormulaCuadratica(filename='problem1.txt')
p3 = FormulaCuadratica(filename='problem1.txt')

ex1 = ExamGenerator(filename='examen.txt')
ex1.add_item('problema1',p1)
ex1.add_item('problema2',p2)
ex1.add_item('problema3',p3)
s1 = ex1.generate_text()
ex1.save()

In [13]:
from IPython.display import display, Markdown
display(Markdown(s1))

Titulo:

1. Dada la ecuación cuadrática $2 x^{2} = 3 - x$. Identifique los valores de a, b, c y resuelva la ecuación usando la formula cuadrática $x = \frac{- b + \sqrt{- 4 a c + b^{2}}}{2 a}$.

End 1

2. Dada la ecuación cuadrática $- 2 x^{2} = 4 x - 8$. Identifique los valores de a, b, c y resuelva la ecuación usando la formula cuadrática $x = \frac{- b + \sqrt{- 4 a c + b^{2}}}{2 a}$.

End 2

3. Dada la ecuación cuadrática $- 10 x^{2} = 8 x + 9$. Identifique los valores de a, b, c y resuelva la ecuación usando la formula cuadrática $x = \frac{- b + \sqrt{- 4 a c + b^{2}}}{2 a}$.

End 3

End Exam